In [1]:
import cv2
import skvideo
import math
import os
import numpy as np
cv2.__version__

'3.4.1'

In [2]:
#libraries
import tensorflow as tf
import tflearn
tf.__version__

/home/oleg/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.10.0'

In [3]:
X_bad = np.load('x_bad.npy')
X_good = np.load('x_good.npy')
print X_good.shape, X_bad.shape

(1594, 20, 32) (1240, 20, 32)


In [4]:
y_bad = np.ones(X_bad.shape[0])

In [5]:
y_good = np.zeros(X_good.shape[0])

In [6]:
X = np.concatenate((X_bad, X_good), axis=0)
X.shape

(2834, 20, 32)

In [7]:
y = np.concatenate((y_bad, y_good))

In [8]:
from sklearn.cross_validation import train_test_split

/home/oleg/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [10]:
X_train.shape, y_train.shape, X_test.shape

((2267, 20, 32), (2267,), (567, 20, 32))

In [12]:
model = tf.keras.Sequential()
#model.add(tf.keras.layers.Reshape((10,2048), input_shape=[10,1,1,2048] ))
model.add(tf.keras.layers.BatchNormalization( input_shape=[20,32],))
model.add(tf.keras.layers.LSTM(8,  
                               dropout=0.2,
                               #kernel_initializer='random_uniform',
                               #activation='sigmoid'
                               return_sequences=True,
                              ))
model.add(tf.keras.layers.LSTM(8, dropout=0.2, ))
#model.add(tf.keras.layers.LSTM(2, recurrent_dropout=0.2, ))
#model.add(tf.keras.layers.LSTM(2, recurrent_dropout=0.2, ))
#model.add(tf.keras.layers.LSTM(1, recurrent_dropout=0.2, ))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 20, 32)            128       
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 8)             1312      
_________________________________________________________________
lstm_3 (LSTM)                (None, 8)                 544       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 1,993
Trainable params: 1,929
Non-trainable params: 64
_________________________________________________________________


In [13]:
adam = tf.keras.optimizers.Adam(lr=1e-2, )
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
with tf.device('/gpu:0'):
    model.fit(X_train, 
              y_train, 
              batch_size=64,  
              shuffle=True,
              epochs=50, 
              verbose=2,
                    #use_multiprocessing=True,
              validation_data=(X_test , y_test)
                     )

Train on 2267 samples, validate on 567 samples
Epoch 1/50
 - 3s - loss: 0.4182 - acc: 0.8337 - val_loss: 0.6219 - val_acc: 0.7725
Epoch 2/50
 - 1s - loss: 0.2581 - acc: 0.9034 - val_loss: 0.2650 - val_acc: 0.9101
Epoch 3/50
 - 1s - loss: 0.2051 - acc: 0.9241 - val_loss: 0.1764 - val_acc: 0.9330
Epoch 4/50
 - 1s - loss: 0.1765 - acc: 0.9338 - val_loss: 0.1755 - val_acc: 0.9489
Epoch 5/50
 - 1s - loss: 0.1629 - acc: 0.9449 - val_loss: 0.1255 - val_acc: 0.9612
Epoch 6/50
 - 1s - loss: 0.1524 - acc: 0.9466 - val_loss: 0.0938 - val_acc: 0.9753
Epoch 7/50
 - 1s - loss: 0.1355 - acc: 0.9550 - val_loss: 0.0810 - val_acc: 0.9753
Epoch 8/50
 - 1s - loss: 0.1322 - acc: 0.9541 - val_loss: 0.1177 - val_acc: 0.9541
Epoch 9/50
 - 1s - loss: 0.1346 - acc: 0.9528 - val_loss: 0.0889 - val_acc: 0.9700
Epoch 10/50
 - 1s - loss: 0.1036 - acc: 0.9669 - val_loss: 0.0780 - val_acc: 0.9753
Epoch 11/50
 - 1s - loss: 0.1008 - acc: 0.9704 - val_loss: 0.0551 - val_acc: 0.9859
Epoch 12/50
 - 1s - loss: 0.0945 - acc

In [15]:
model.save('video_clasification2.h5')